# 🏛️ Agent Architecture Demo

This notebook visualizes the internal structure of the LangGraph agent and demonstrates its execution flow.

## Features
- **Graph Visualization**: See the nodes and edges of the agent
- **Step-by-Step Execution**: Trace how the agent processes a query
- **State Inspection**: View the agent's internal state at each step

---

In [ ]:
# === COLAB SETUP (Auto-detects environment) ===
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("🔧 Running in Google Colab - Setting up environment...")
    
    # Clone repository
    !git clone https://github.com/MasumRab/gemini-fullstack-langgraph-quickstart --depth 1
    %cd gemini-fullstack-langgraph-quickstart/backend
    
    # Clean conflicting packages
    !pip uninstall -y google-ai-generativelanguage google-generativeai tensorflow grpcio-status 2>/dev/null || true
    
    # Install backend dependencies
    !pip install -q .
    
    # Set API Key from Colab secrets
    import os
    from google.colab import userdata
    try:
        os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
        print("✅ API Key loaded from Colab Secrets")
    except:
        print("⚠️ API Key not found in secrets. Please set it manually:")
        os.environ["GEMINI_API_KEY"] = input("Enter your Gemini API Key: ")
else:
    print("💻 Running locally - Assuming dependencies are installed")
    import os
    import sys
    # Add backend/src to path for local development
    sys.path.insert(0, os.path.abspath("../backend/src"))

In [ ]:
# --- MODEL CONFIGURATION ---
# @title Select Gemini Model
# @markdown Choose the model strategy to use. Use 'Free Tier' if you encounter Rate Limits.

MODEL_STRATEGY = "Free Tier (Gemini 1.5 Flash)" # @param ["Free Tier (Gemini 1.5 Flash)", "Experimental (Gemini 2.0 Flash)", "Flash Lite (Gemini 2.0 Flash-Lite)"]

import os

# Map selection to actual model names
if MODEL_STRATEGY == "Free Tier (Gemini 1.5 Flash)":
    SELECTED_MODEL = "gemini-1.5-flash"
    print("✅ Using Free Tier Model: gemini-1.5-flash")
elif MODEL_STRATEGY == "Experimental (Gemini 2.0 Flash)":
    SELECTED_MODEL = "gemini-2.0-flash-exp"
    print("⚠️ Using Experimental Model: gemini-2.0-flash-exp (Quota Required)")
elif MODEL_STRATEGY == "Flash Lite (Gemini 2.0 Flash-Lite)":
    SELECTED_MODEL = "gemini-2.0-flash-lite-preview-02-05"
    print("⚠️ Using Preview Model: gemini-2.0-flash-lite-preview-02-05 (Quota Required)")

# Set Environment Variables to override defaults
os.environ["QUERY_GENERATOR_MODEL"] = SELECTED_MODEL
os.environ["REFLECTION_MODEL"] = SELECTED_MODEL
os.environ["ANSWER_MODEL"] = SELECTED_MODEL
os.environ["TOOLS_MODEL"] = SELECTED_MODEL

## 1. Load the Agent Graph

We import the compiled graph object from the backend code.

In [ ]:
# Import the graph
try:
    from agent.graph import graph
    print("✅ Graph imported successfully")
except ImportError as e:
    print(f"❌ Failed to import graph: {e}")
    print("Ensure you are in the correct directory (backend/) and dependencies are installed.")

## 2. Visualize the Architecture

We use Mermaid.js to render the graph structure directly in the notebook.

In [ ]:
from IPython.display import Image, display

print("📊 Generating Graph Visualization...")

try:
    # Draw the graph as a PNG
    png_data = graph.get_graph().draw_mermaid_png()
    display(Image(png_data))
    print("✅ Visualization complete")
except Exception as e:
    print(f"⚠️ Graphical visualization failed: {e}")
    print("\nFalling back to ASCII representation:")
    print("="*40)
    graph.get_graph().print_ascii()
    print("="*40)

## 3. Run a Demo Query

Execute a simple query to see the agent in action.

In [ ]:
from langchain_core.messages import HumanMessage
import asyncio

async def run_demo(question: str):
    """Run the agent and stream output."""
    print(f"🤖 Processing Query: \"{question}\"")
    print("-" * 50)

    config = {"configurable": {"thread_id": "demo_notebook"}}
    
    async for event in graph.astream_events(
        {"messages": [HumanMessage(content=question)]},
        config,
        version="v2"
    ):
        kind = event["event"]
        
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end="", flush=True)
                
        elif kind == "on_tool_start":
            print(f"\n\n🛠️ Executing Tool: {event['name']}")
            
        elif kind == "on_tool_end":
            print(f"✅ Tool Completed: {event['name']}\n")

    print("\n" + "-" * 50)
    print("🏁 Execution Complete")

# Run the demo
QUERY = "What is the capital of France?"
await run_demo(QUERY)

---
## 📚 Learn More

- **Visualization Guide**: `docs/guides/VISUALIZATION_EXAMPLES.md`
- **LangGraph Docs**: [https://langchain-ai.github.io/langgraph/](https://langchain-ai.github.io/langgraph/)